# Options Arbitrage in Imperfect Markets

## Stephen Figlewski

<p style="text-align: center;"><B>ABSTRACT</B></p>
Option valuation models are based on an arbitrage strategy-hedging the option against
the underlying asset and rebalancing continuously until expiration that is only possible
in a frictionless market. This paper simulates the impact of market imperfections and
other problems with the "standard" arbitrage trade, including uncertain volatility,
transactions costs, indivisibilities, and rebalancing only at discrete intervals. We find
that, in an actual market such as that for stock index options, the standard arbitrage is
exposed to such large risk and transactions costs that it can only establish very wide
bounds on equilibrium options prices. This has important implications for price determination in options markets, as well as for testing of valuation models.

## Summary

Figlewski uses simulated data to explore the ramifications of BSM assumptions failing. He does this by simulating one month of 250 fictional assets and then examines the performance of options trading strategies under the following conditions:
- Incorrect volatility estimates
- Indivisible assets (hedge ratio must be rounded to full shares)
- Transaction Costs
    - discontinuous rebalancing as a result of transaction costs and times discrete nature.

This strategy was relatively novel at the time, and remains highly informative about the power of simulation for answering questions without a closed form solution.

## Catallactic Interpretation

This simulation strategy is a highly catallactic approach. That utilizes data from the as-if world proposed in the model to explore how subjective beliefs such as volatility estimates or risk aversion impact an agents decisions and how those influence prices.

## Experimental Design

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from math import exp, log, sqrt, abs

Figlewski simulated 25 days of price data for 250 assets that each had an initial value of 100.00. Each daily price after the first was calculated using the formula: 
$$P_{t+1} = P_t*e^{R+vz}$$
Where R is the mean rate of price change per day, v is the daily volatility, and z is a draw from the standard normal distribution. R and v are set to the daily equivalent of 15% and 0.15 i.e.
$$R = \frac{log(1.15)}{260} = 0.000538$$
$$v = \frac{0.15}{260^{0.5}} = 0.0093$$
The following code replicates this process:

In [6]:
ar=0.15

R=log(1+ar)/260
v=ar/sqrt(260)

assets = np.empty((250,25))
daily_returns = np.empty((250,25))
for i in range(250):
    price=np.empty(25)
    price[0]=100
    dr=np.empty(25)
    dr[0]=0
    for t in range(1,25):
        price[t]=price[t-1]*exp(R+v*np.random.normal(0,1))
        dr=log(price[t]/price[t-1])
    assets[i]=price
    daily_returns[i]=dr

final_prices = assets[:,-1]
rates_of_return = ((final_prices/assets[:,0])-1)*100
mean_annualized_RoR = np.mean(rates_of_return)*(260/24)
stdev_annualized_RoR = np.std(rates_of_return)*sqrt(260/24)

vol=np.empty(250)
for i in range(250):
    vol[i]=np.std(daily_returns[i])

mean_annualized_volatility=np.mean(vol)*sqrt(260/24)
stdev_annualized_volatility=np.std(vol)*sqrt(260/24)

                    Mean      Std. Dev  
Final Price:        101.89    4.83 
Rate of Return:     20.46     15.89
Volatility:         0.000     0.000


In [15]:
print(f"{'' : <20}{'Mean' : <10}{'Std. Dev' : <10}")
print(u'\u2500' * 40)
print(f"{'Final Price:': <20}{np.mean(final_prices):<10.2f}{np.std(final_prices):<5.2f}")
print(f"{'Rate of Return:': <20}{mean_annualized_RoR:<10.2f}{stdev_annualized_RoR:.2f}")
print(f"{'Volatility:': <20}{mean_annualized_volatility:<10.3f}{stdev_annualized_volatility:.3f}")

                    Mean      Std. Dev  
────────────────────────────────────────
Final Price:        101.89    4.83 
Rate of Return:     20.46     15.89
Volatility:         0.000     0.000


Using the simulated prices Figlewski then calculates the BSM option price and delta hedging ratio at four strike prices (91,100,103,105) using a risk-free interest rate of 5% and the true annual volatility of 0.15.

In [29]:
## Define method to calculate the price of a European call option using Black-Scholes model
def bscall(price,strike,rfr,vol,div,T):
    N=stats.norm.cdf
    d1=(np.log(price/strike)+(rfr-div+vol**2/2)*T)/(vol*np.sqrt(T))
    d2=d1-vol*np.sqrt(T)
    call = price*np.exp(-div*T)*N(d1)-strike*np.exp(-rfr*T)*N(d2)
    delta=N(d1)
    return call,delta

In [9]:
rfr=0.05
div=0
vol=0.15

strikes=[97,100,103,105]
option_prices=np.empty((4,250,25))
call_deltas=np.empty((4,250,25))
for k in range(4):
    for i in range(250):
        calls=np.empty(25)
        deltas=np.empty(25)
        for j in range(25):
            T=(25-j)
            price=assets[i,j]
            calls[j], deltas[j]=bscall(price,strikes[k],rfr,vol,div,T/260)
        option_prices[k][i]=calls
        call_deltas[k][i]=deltas

avg_option_prices=np.empty(4)
stdev_option_prices=np.empty(4)
for i in range(4):
    avg_option_prices[i]=np.mean(option_prices[i])
    stdev_option_prices[i]=np.std(option_prices[i])

inTheMoney = np.empty((4,250,2))
for i in range(4):
    for p in range(250):
        if final_prices[p]>strikes[i]:
            inTheMoney[i][p][0]=1
            inTheMoney[i][p][1]=final_prices[p]-strikes[i]
        else:
            inTheMoney[i][p][0]=0
            inTheMoney[i][p][1]=0


avg_ITM=np.empty(4)
stdev_ITM=np.empty(4)
num_ITM=np.empty(4)
for i in range(4):
    avg_ITM[i]=np.mean(inTheMoney[i,:,1]*inTheMoney[i,:,0])
    stdev_ITM[i]=np.std(inTheMoney[i,:,0]*inTheMoney[i,:,1])
    num_ITM[i]=np.sum(inTheMoney[i,:,0])

In [19]:

print(f"{'': <10}{'Initial':^10}{'Final Value':^20}{'Overall':^20}{'In The Money':^32}")
print(f"{'Strike':<10}{'Price':^10}{'Mean':^10}{'Std. Dev':^10}{'Mean':^10}{'Std. Dev':^10}{'Number':^10}{'Avg. Amount':^12}{'Std. Dev':^10}")
print(u'\u2500' * 92)
for i in range(4):
    print(f'{strikes[i]: <10}{np.mean(option_prices[i,:,0]):^10.2f}{np.mean(option_prices[i,:,-1]):^10.2f}{np.std(option_prices[i,:,-1]):^10.2f}{avg_option_prices[i]:^10.2f}{stdev_option_prices[i]:^10.2f}{num_ITM[i]:^10}{avg_ITM[i]:^12.2f}{stdev_ITM[i]:^10.2f}')

           Initial      Final Value           Overall                 In The Money          
Strike      Price      Mean    Std. Dev    Mean    Std. Dev   Number  Avg. Amount  Std. Dev 
────────────────────────────────────────────────────────────────────────────────────────────
97           4.06      5.33      4.16      4.57      2.80     206.0       5.30       4.18   
100          2.10      3.11      3.34      2.48      2.17     165.0       3.07       3.36   
103          0.89      1.47      2.34      1.10      1.43     112.0       1.42       2.35   
105          0.44      0.77      1.69      0.56      0.99      67.0       0.72       1.69   


## Market Imperfections and Risk

In order to determine the impacts of agents and market makers having different volatility estimates. Figlewski simulated the call pricing from the market makers perspective using 0.10, 0.13, 0.15, 0.17, and 0.20 as the market makers expected volatility. Figlewski then examines the resulting arbitrage opportunities that are presented by these differing priors.

In [40]:
volatilities=np.array([0.1,0.13,0.15,0.17,0.2])
agent_prices=np.empty(4)
agent_deltas=np.empty(4)
for i in range(4):
    agent_prices[i], agent_deltas[i]= bscall(100,strikes[i],rfr,0.15,div,25/260)

print(f"{'': <45}{'Trading Strategy':^20}{'':^10}{'Excess Return':^20}")
print(f"{'Strike':<10}{'Market maker':^15}{'Call':^10}{'Hedge':^10}{'':^20}{'Cost of RL':^10}")
print(f"{'Price':<10}{'Volatility':^15}{'Price':^10}{'Ratio':^10}{'Call':^10}{'Stock':^10}{'Position':^10}{'$ amount':^10}{'Annual %':^10}")
print(u'\u2500' * 95)
output="{strike:<10}{vol:^15.2f}{price:^10.2f}{delta:^10.3f}{pos:^20}{position:^10.2f}{amount:^10.3f}{annual:^10.2f}"
for k in range(4):
    
    for v in range(5):
        price, delta = bscall(100,strikes[k],rfr,volatilities[v],div,25/260)
        if price>agent_prices[k]:
            trade='Sell     Buy'
        elif price<agent_prices[k]:
            trade='Buy     Sell'
        else:
            trade='No Trade'

        if trade=='Buy     Sell':
            rlp=price-agent_deltas[k]*100
        elif trade=='Sell     Buy':
            rlp=agent_deltas[k]*100-price
        else:
            rlp=agent_deltas[k]*100-price
        profit=abs(agent_prices[k]-price)
        ann_profit=(profit)*260/25
        if v==0:
            print(output.format(strike=strikes[k],vol=volatilities[v],price=price,delta=delta,pos=trade,position=rlp,amount=profit,annual=ann_profit))
        else:
            print(output.format(strike=' ',vol=volatilities[v],price=price,delta=delta,pos=trade,position=rlp,amount=profit,annual=ann_profit))
    print()
 

                                               Trading Strategy               Excess Return    
Strike     Market maker     Call     Hedge                       Cost of RL
Price       Volatility     Price     Ratio      Call     Stock    Position  $ amount  Annual % 
───────────────────────────────────────────────────────────────────────────────────────────────
97             0.10         3.66     0.876       Buy     Sell      -74.61    0.397      4.13   
               0.13         3.88     0.815       Buy     Sell      -74.39    0.174      1.81   
               0.15         4.06     0.783         No Trade        74.22     0.000      0.00   
               0.17         4.24     0.757       Sell     Buy      74.03     0.189      1.96   
               0.20         4.55     0.726       Sell     Buy      73.73     0.490      5.10   

100            0.10         1.49     0.568       Buy     Sell      -53.55    0.612      6.36   
               0.13         1.86     0.555       Buy     Se

Whether the agent's or market maker's volatility is closer to reality is irrelevant, what this shows us is a framework for introducing subjective beliefs into a neoclassical model. Figlewski mentions that even if the true long term volatility is 0.15, the volatility during the contract period of the option may be anything. 

### Excess Returns

The next experiment that was run showed the standard deviations of excess returns when incorrect volatility is used, hedges are only rebalanced daily, and hedge ratios are rounded to the nearest multiple of K. Excess returns can be calculated as:
$$ER_t = (C_t - C_{t-1}) - h_{t-1}*(P_t - P_{t-1}) - r*(C_{t-1} - h_{t-1}*P_{t-1})$$
Where C is the price of the call option, P is the price of the underlying, h is the delta-hedging ratio, and r is the daily interest rate.

In [57]:
def excess_return(option_prices, call_deltas, assets, rfr):    
    d_excess_return= np.empty((4,250,25))
    d_excess_return[:,:,0]=0
    for i in range(4):
        for j in range(250):
            for k in range(1,25):
                ct=option_prices[i][j][k]
                ct1=option_prices[i][j][k-1]
                h=call_deltas[i][j][k-1]
                pt=assets[j][k]
                pt1=assets[j][k-1]
                dr=rfr/260
                d_excess_return[i][j][k]=(ct-ct1)-h*(pt-pt1)-dr*(ct1-h*pt1)

    e_r=np.empty((4,250))
    for i in range(4):
        for j in range(250):
            e_r[i][j]=np.sum(d_excess_return[i][j])
    return e_r

In [58]:
volatilities=np.array([0.1,0.13,0.15,0.17,0.2])
option_prices=np.empty((5,4,250,25))
call_deltas=np.empty((5,4,250,25))

for v in range(5):
    for k in range(4):
        for i in range(250):
            for j in range(25):
                T=(25-j)
                price=assets[i,j]
                option_prices[v][k][i][j], call_deltas[v][k][i][j]=bscall(price,strikes[k],rfr,volatilities[v],div,T/260)

K=[0.02,0.05,0.10,0.25,1]
excess_returns=np.empty((10,4,250))



In [59]:
for i in range(5):
    excess_returns[i]=excess_return(option_prices[i], call_deltas[i], assets, rfr)

for i in range(5):
    r_deltas=K[i] * (call_deltas/ K[i]).round()
    excess_returns[i+5]=excess_return(option_prices[2], r_deltas[2], assets, rfr)

In [60]:
er_std = np.empty((10,4))
for i in range(10):
    for j in range(4):
        er_std[i][j]=excess_returns[i][j].std()

In [61]:
print(f"{'': <25}{'X = 97': ^10}{'X = 100':^10}{'X = 103':^10}{'X = 105':^10}")
print(u'\u2500' * 65)
print(f"{'Base Case':<25}{er_std[2][0]:^10.2f}{er_std[2][1]:^10.2f}{er_std[2][2]:^10.2f}{er_std[2][3]:^10.2f}")
print('Incorrect Volatility')
for i in range(5):
    print(f"v = {volatilities[i]:<21.2f}{er_std[i][0]:^10.2f}{er_std[i][1]:^10.2f}{er_std[i][2]:^10.2f}{er_std[i][3]:^10.2f}")

print('Indivisibilities')
for i in range(5,10):
        print(f"K = {K[i-5]:<21.2f}{er_std[i][0]:^10.2f}{er_std[i][1]:^10.2f}{er_std[i][2]:^10.2f}{er_std[i][3]:^10.2f}")

                           X = 97   X = 100   X = 103   X = 105  
─────────────────────────────────────────────────────────────────
Base Case                   0.23      0.30      0.27      0.24   
Incorrect Volatility
v = 0.10                    0.36      0.40      0.41      0.39   
v = 0.13                    0.25      0.32      0.29      0.27   
v = 0.15                    0.23      0.30      0.27      0.24   
v = 0.17                    0.24      0.30      0.28      0.26   
v = 0.20                    0.30      0.33      0.33      0.33   
Indivisibilities
K = 0.02                    0.23      0.30      0.27      0.24   
K = 0.05                    0.23      0.30      0.27      0.25   
K = 0.10                    0.24      0.33      0.30      0.26   
K = 0.25                    0.36      0.43      0.40      0.37   
K = 1.00                    0.90      1.39      1.39      0.96   


![img info](tableIII.png)

## Transactions Costs

The previous section showed that while introducing market imperfections increased the variance of outcomes, it did not bias the returns one way or the other. Transaction costs on the other hand reduce the profitability of every strategy.

![img](tableIV.png)

Figlewski explores how these transaction costs would reduce the number of hedging updates that can be made while still maintaining profitability.
Two common strategies for approaching this are tested and their results shown. Rebalancing every K days, and rebalancing when the hedge ratio changes by a specified amount.

## Arbitrage Bounds Based on the Standard Arbitrage

This section is perhaps the most catallactic in my opinion. Table V shows the bid-ask spread of options with a 50% or 75% probability of covering costs. Taking these bid-ask spread as a subjective valuation of the asset, or a measure of risk tolerance allows us to step outside the no-trade theorem while being inside a Black-Scholes world!

![img](tableV.png)

## Other Option Strategies

The remainder of the paper explores similar ideas with different asset types, however this portion is less key to the interpretation I aim to showcase.

![img](tableVI.png)